In [7]:
from transformers import AutoConfig, AutoModelForSeq2SeqLM, AutoTokenizer

# Download configuration from huggingface.co and cache.
# model_id = "facebook/mbart-large-50-many-to-many-mmt"
model_id = "google-t5/t5-base"

# does not download the pretrained weights, just affects configuration
# use AutoModelForSeq2SeqLM.from_pretrained to also download the weights
config = AutoConfig.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_config(config)

c:\Users\abhiv\miniconda3\envs\mse244\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# now, need to add a tokenizer and embedding layer to the top of the model
auto_tk = AutoTokenizer.from_pretrained(model_id)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

c:\Users\abhiv\miniconda3\envs\mse244\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abhiv\.cache\huggingface\hub\models--google-t5--t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [138]:
from tokenizers import Tokenizer, decoders
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace, ByteLevel
from datasets import load_dataset

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

dataset = load_dataset("csv", data_files="data/v3.csv")
obolo_data = dataset['train']['Obolo']

tokenizer.pre_tokenizer = ByteLevel()
tokenizer.decoder = decoders.ByteLevel()
tokenizer.train_from_iterator(obolo_data, trainer)

In [153]:
print(tokenizer.get_vocab_size())
unique_words_verse = [set(verse.split()) for verse in obolo_data]
unique_words = set()
for s in unique_words_verse:
    unique_words.update(s)
print(len(unique_words))

16463
32083


In [126]:
encoded_inputs = tokenizer.encode_batch(obolo_data[:5])
for enc in encoded_inputs:
    print(enc.tokens)

['Ġme', 'Ġibebene', ',', 'Ġawaji', 'Ġi', 'ÌĤ', 'rom', 'Ġisinyo', 'Ì£', 'n', 'ÌĦ', 'Ġme', 'ÌĢ', 'Ġlinyo', 'Ì£', 'n', 'ÌĦ.']
['Ġire', ',', 'Ġlinyo', 'Ì£', 'n', 'ÌĦ', 'Ġya', 'Ġi', 'ÌĢ', 'kabet', 'Ġlek', 'Ġinu', 'Ġgeege', ',', 'Ġi', 'ÌĢ', 'kpo', 'Ì£', 'kpo', 'Ì£', 'Ġilaak', 'Ġukpo', 'Ì£', 'k', '.', 'Ġmun', 'ÌĦ', 'Ġmun', 'ÌĦ', 'Ġojot', 'Ġlinyo', 'Ì£', 'n', 'ÌĦ.', 'Ġudun', 'Ġokakana', 'Ġotutuuk', 'Ġere', 'Ġichit', 'Ġbu', 'ÌĢ', 'k', '.', 'Ġekwukwu', 'Ġawaji', 'Ġokekiwulu', 'Ġefe', 'Ġigba', 'Ġiyak', 'Ġikana', 'Ġme', 'Ġinyo', 'Ì£', 'n', 'ÌĦ', 'Ġisi', 'Ġmun', 'ÌĦ.']
['Ġawaji', 'Ġonenitumu', 'Ġibe', ',', 'ĠâĢľ', 'utoon', 'ÌĦ', 'Ġikup', '!âĢĿ', 'Ġutoon', 'ÌĦ', 'Ġokukup', '.']
['Ġawaji', 'Ġomumun', 'ÌĦ', 'Ġibe', 'Ġke', 'Ġutoon', 'ÌĦ', 'Ġya', 'Ġi', 'ÌĤ', 'jaan', 'ÌĦ,', 'Ġme', 'ÌĢ', 'Ġiniche', 'Ġutoon', 'ÌĦ', 'Ġya', 'Ġisan', 'ÌĦ', 'a', 'Ġme', 'Ġlek', 'Ġudun', 'Ġya', ',']
['Ġme', 'ÌĢ', 'Ġigwen', 'Ġutoon', 'ÌĦ', 'Ġya', 'Ġegwe', ',', 'Ġme', 'ÌĢ', 'Ġinigwen', 'Ġudun', 'Ġya', 'Ġeririeen', 'ÌĦ.', 'Ġeririee

In [127]:
tokenizer.save('data/obolo-bpe-tokenizer.json')
# tokenizer = Tokenizer.from_file('data/obolo-bpe-tokenizer.json')

In [128]:
out = tokenizer.encode(obolo_data[100])
print(out.tokens)
print(tokenizer.decode(out.ids))

['Ġjaban', 'Ġonenikaan', 'ÌĦ', 'Ġngwan', 'ÌĦ', 'Ġenerieen', 'ÌĦ', 'Ġge', 'Ġekigwen', 'Ġjuban', '.', 'Ġjuban', 'Ġore', 'Ġadasi', 'Ġene', 'Ġo', 'ÌĢ', 'bebene', 'Ġikikwak', 'Ġuneen', 'ÌĦ,', 'Ġme', 'ÌĢ', 'Ġikiwut', 'Ġo', 'ÌĢ', 'ja', '.']
 jaban onenikaan̄ ngwan̄ enerieen̄ ge ekigwen juban. juban ore adasi ene òbebene ikikwak uneen̄, mè ikiwut òja.


In [131]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file='data/obolo-bpe-tokenizer.json')
fast_tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=16463, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [136]:
ids = fast_tokenizer.encode("mè ekekpulu me egwe mè eririeen̄, inyi ekeche utoon̄ esan̄a me lek udun. awaji okpọkpọ, mè imun̄ ibe ke inu cha îjaan̄.")
fast_tokenizer.decode(ids)

' mè ekekpulu me egwe mè eririeen̄, inyi ekeche utoon̄ esan̄a me lek udun. awaji okpọkpọ, mè imun̄ ibe ke inu cha îjaan̄.'

In [72]:
auto_tk.batch_encode_plus(obolo_data[:5])['input_ids'][0]

[140,
 3,
 23,
 346,
 15719,
 6,
 3,
 9,
 210,
 17815,
 1889,
 3522,
 19,
 77,
 63,
 2,
 29,
 2,
 3,
 51,
 5115,
 3,
 40,
 77,
 63,
 2,
 29,
 2,
 5,
 1]